### Use this notebook for developing mlframe prototype 2

In [27]:
import multiprocessing
from Queue import Queue, Empty
import subprocess
import os
import time
import re

In [37]:
# Class for executing commands in background processes.
# command_and_args can be a list or a string with the command and arguments separated with spaces.
# It can be used for executing remote processes in background. For that purpose exec_remote.sh should be used.
# Usage sample: 
# d = BashExecutor(command,hostname=hostname)
# d.start()
# exec_remote.sh must output subprocess exit code in the form:
# exitcode=N
# , where N is the number.
class BashExecutor(multiprocessing.Process):
    debug = False
    exitcode=None
    
    def __init__(self, command_and_args, debug=False, hostname=""):
        super(BashExecutor,self).__init__()
        self.debug = debug
        self.hostname = hostname
        if self.debug: print "command args:",command_and_args
        if type(command_and_args) is str:
            self.command = command_and_args.split(" ")
        else:
            self.command=command_and_args
        self.stdout=Queue()
        self.stderr=Queue()
        self.exitcode_pat = re.compile("^exitcode=(\d+)")
        #if self.debug: print "self.command=",self.command
        #self.daemon = True
        
    # Poll exit code of self.proc and store it if not None.
    def poll(self):
        exitcode = self.proc.poll()
        if exitcode is not None:
            # Set exticode only if subprocess didn't set it __handleOutput__
            if self.exitcode is None:
                self.exitcode = exitcode
        return exitcode
    
    # Copy process stdout to self.stdout queue line by line.
    def __handleOutput__(self):
        if self.debug: print self.name, "Handle output"
        proc = self.proc
        for line in iter(proc.stdout.readline, b''):
            m = self.exitcode_pat.match(line)
            if m is not None:
                print "Exitcode match",m.group(0)
            self.stdout.put(line)
            if line is not None and len(line)>0:
                    print self.hostname,line,
                    
    # Copy process stderr to self.stderr queue line by line.
    def __handleError__(self):
        if self.debug: print self.name, "Handle error"
        proc = self.proc
        for line in iter(proc.stderr.readline, b''):
            self.stderr.put(line)
            if line is not None and len(line)>0:
                    print "!",self.hostname,line,
        
    # Must be calles to start the process in background
    def run(self):
        if self.debug: 
            print "In {}. Calling {}".format(self.name,self.command)
            
        proc = subprocess.Popen(self.command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, bufsize=1, shell=False)
        self.proc = proc
        p_out = multiprocessing.Process(target=self.__handleOutput__)
        p_out.start()
        p_err = multiprocessing.Process(target=self.__handleError__)
        p_err.start()
        if self.debug: print self.name, "process started"
        while self.poll() is None:
            try:
                line = self.stdout.get(block=False,timeout=1)
                if self.debug: 
                    if line is not None:
                        print self.name,"stdout:",line
            except Empty:
                time.sleep(1)
            
        #self.__handleOutput__(proc)
        if self.debug:
            print "Exiting",multiprocessing.current_process().name,self.exitcode
        return

    

In [38]:
# Execute command on remote (or local) host in background, using exec_remote.sh and BashExecutor instance.
# Command can be any command or executable file name.
# Arguments must be in a string separated with spaces.
def RemoteExec(host, command_and_args, debug=False):    
    if debug: print "RemoteExec commands and args:",command_and_args
    CA = command_and_args.split(" ")
    if debug: print CA
    command = CA[0]
    args = CA[1:]
    if debug: print "Command:",command
    if debug: print "Args:",args
    package_directory = os.path.dirname(os.getcwd())
    scripts_location=os.path.realpath(os.path.join(package_directory,"mlframe","scripts"))
    exec_remote_script="exec_remote.sh"
    script_path = os.path.join(scripts_location,exec_remote_script)
    
    # Merge args into a string
    command_script_path = os.path.join(scripts_location, command)
    if debug: print "Test if script file exists in RemoteExec",command_script_path
    if os.path.isfile(command_script_path):
        command = command_script_path
    if debug: print "Command:",command
          
    if host == "" or host == "localhost" or host == "127.0.07":
        command_args_list = [command] + args
    else:
        command_args_list = [script_path] + [host] + [command] + args
    if debug: print "Calling BashExecutor with args",command_args_list
    d = BashExecutor(command_args_list, hostname=host, debug=debug)
    d.start()

    #for d in jobs:
    #    d.join()
    
    if debug: print "finished",host,command

In [40]:
RemoteExec("localhost","test.sh",debug=True)
RemoteExec("mouse","test_nvidia_ubuntu.sh", debug=False)
RemoteExec("DL","test_nvidia_ubuntu.sh", debug=False)

RemoteExec commands and args: test.sh
['test.sh']
Command: test.sh
Args: []
Test if script file exists in RemoteExec /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
Command: /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
Calling BashExecutor with args ['/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh']
command args: ['/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh']
finished localhost /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
In BashExecutor-54. Calling ['/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh']
BashExecutor-54 Handle output
BashExecutor-54 Handle error
BashExecutor-54 process started
localhost Running test command on peters-imac.local
localhost peters-imac.local():1
! mouse Warning: Permanently added 'mouse.local,fe80::e27:b843:c6d6:6e6f%en1' (ECDSA) to the list of known hosts.
mouse remote_command_.sh arguments: 
mouse NVDRV:384.111,CUDA:9.0,cuDNN:7.0.5.15-1
Ex